In [1]:
%run basics
%matplotlib

Using matplotlib backend: Qt4Agg


In [2]:
cf = {"Files":{},"Options":{},"NEE":{},"GPP":{},"ER":{}}
cf["Files"]["plot_path"] = "../Sites/Whroo/Plots/"
cf["Files"]["file_path"] = "../Sites/Whroo/Data/Portal/"
cf["Files"]["out_filename"] = "Whroo_L6_All.nc"
cf["Options"]["call_mode"] = "interactive"

In [3]:
ofqc_name = "../Sites/Whroo/Data/Portal/Whroo_L6.nc"
rep_name = "../Sites/Whroo/Data/REddyProc/Whroo_L6.nc"
dgo_name = "../Sites/Whroo/Data/DINGO/V12a/Whroo_L6.nc"

In [4]:
bd = {"OzFluxQC":{},"REddyProc":{},"DINGO":{}}
bd["OzFluxQC"]["ds"] = qcio.nc_read_series(ofqc_name)
bd["REddyProc"]["ds"] = qcio.nc_read_series(rep_name)
bd["DINGO"]["ds"] = qcio.nc_read_series(dgo_name)

In [5]:
ofqc_gpp_list = [item for item in bd["OzFluxQC"]["ds"].series.keys() if "GPP" in item]
ofqc_nee_list = [item for item in bd["OzFluxQC"]["ds"].series.keys() if "NEE" in item]
ofqc_er_list = [item for item in bd["OzFluxQC"]["ds"].series.keys() if "ER" in item]
type_list = []
for item in ofqc_er_list:
    type_list.append(item[2:])
for item in type_list:
    if "NEE"+item not in ofqc_nee_list:
        if "ER"+item in ofqc_er_list: ofqc_er_list.remove("ER"+item)
        if "GPP"+item in ofqc_gpp_list: ofqc_gpp_list.remove("GPP"+item)
    if "GPP"+item not in ofqc_gpp_list:
        if "ER"+item in ofqc_er_list: ofqc_er_list.remove("ER"+item)
        if "NEE"+item in ofqc_nee_list: ofqc_nee_list.remove("NEE"+item)
for item in ofqc_nee_list:
    cf["NEE"][item] = {}
for item in ofqc_gpp_list:
    cf["GPP"][item] = {}
for item in ofqc_er_list:
    cf["ER"][item] = {}
rep_gpp_list = [item for item in bd["REddyProc"]["ds"].series.keys() if "GPP" in item]
rep_nee_list = [item for item in bd["REddyProc"]["ds"].series.keys() if "NEE" in item]
rep_er_list = [item for item in bd["REddyProc"]["ds"].series.keys() if "ER" in item]
bd["REddyProc"]["series_list"] = rep_gpp_list+rep_nee_list+rep_er_list
dgo_gpp_list = [item for item in bd["DINGO"]["ds"].series.keys() if "GPP" in item]
dgo_nee_list = [item for item in bd["DINGO"]["ds"].series.keys() if "NEE" in item]
dgo_er_list = [item for item in bd["DINGO"]["ds"].series.keys() if "ER" in item]
bd["DINGO"]["series_list"] = dgo_gpp_list+dgo_nee_list+dgo_er_list

In [6]:
ds_ofqc = bd["OzFluxQC"]["ds"]
nRecs = int(ds_ofqc.globalattributes["nc_nrecs"])
dt_ofqc = ds_ofqc.series["DateTime"]["Data"]
start_date = dt_ofqc[0]
end_date = dt_ofqc[-1]
for item in ["REddyProc","DINGO"]:
    ds = bd[item]["ds"]
    ts = ds.globalattributes["time_step"]
    dt = ds.series["DateTime"]["Data"]
    si = qcutils.GetDateIndex(dt,str(start_date),ts=ts,default=0,match="exact")
    ei = qcutils.GetDateIndex(dt,str(end_date),ts=ts,default=len(dt)-1,match="exact")
    dt = dt[si:ei+1]
    index = qcutils.FindIndicesOfBInA(dt,dt_ofqc)
    for label in bd[item]["series_list"]:
        data = numpy.ma.ones(nRecs)*float(c.missing_value)
        flag = numpy.ma.ones(nRecs)
        data_import,flag_import,attr_import = qcutils.GetSeriesasMA(ds,label,si=si,ei=ei)
        data[index] = data_import
        flag[index] = flag_import
        if "NEE" in label: cf["NEE"][label+"_"+item] = {}
        if "GPP" in label: cf["GPP"][label+"_"+item] = {}
        if "ER" in label: cf["ER"][label+"_"+item] = {}
        qcutils.CreateSeries(ds_ofqc,label+"_"+item,data,Flag=flag,Attr=attr_import)

In [7]:
qcrp.L6_summary(cf,ds_ofqc)

/home/peter/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:3900: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")
